<a href="https://colab.research.google.com/github/michaelwaheb/CIT690E-DeepLearning_Michael_Reda_191002/blob/main/Copy_of_MaLstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tqdm


In [2]:
from time import time
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import re
!pip install --user -U nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
# from keras.optimizers import Adadelta
from tensorflow import keras
from tensorflow.keras.optimizers import Adadelta
from keras.callbacks import ModelCheckpoint
from tqdm import tqdm

In [3]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle competitions download quora-question-pairs -f data?select=test.csv.zip



mkdir: cannot create directory ‘/root/.kaggle’: File exists
404 - Not Found


In [4]:
! kaggle competitions download quora-question-pairs -f test.csv.zip
! kaggle competitions download quora-question-pairs -f train.csv.zip

! unzip -o test.csv.zip
! unzip -o train.csv.zip

test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  train.csv.zip
  inflating: train.csv               


In [5]:
!pip install wget

import wget
url = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
filename = wget.download(url)

In [6]:
# File paths
#https://www.kaggle.com/c/quora-question-pairs/data?select=test.csv.zip
TRAIN_CSV = './train.csv'
TEST_CSV = './test.csv'
EMBEDDING_FILE = './GoogleNews-vectors-negative300.bin.gz'
MODEL_SAVING_DIR = '/home/ecohen/HDD/HDD4/Models/Kaggle/Quora/'

In [7]:
# Load training and test set
train_df = pd.read_csv(TRAIN_CSV)
test_df = pd.read_csv(TEST_CSV)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
type(train_df)

pandas.core.frame.DataFrame

In [9]:
train_df

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [10]:
test_df

,test_id,question1,question2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?
2,2,What but is the best way to send money from Ch...,What you send money to China?
3,3,Which food not emulsifiers?,What foods fibre?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?
...,...,...,...
3563470,2345791,How do Peaks (TV series): Why did Leland kill ...,What is the most study scene in twin peaks?
3563471,2345792,"What does be ""in transit"" mean on FedEx tracking?",How question FedEx packages delivered?
3563472,2345793,What are some famous Romanian drinks (alcoholi...,Can a non-alcoholic restaurant be a huge success?
3563473,2345794,What were the best and worst things about publ...,What are the best and worst things examination...


In [11]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
stops = set(stopwords.words('english'))
stops

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [13]:
def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text


In [14]:
  # Example 
  sample = "I am Zeinab"
  sample = re.sub(r"Zeinab", "i", sample)
  sample

'I am i'

In [15]:
# Example 
text_to_word_list("How do I out get rid of n't Erectile Dysfunction?")

['how', 'do', 'i', 'out', 'get', 'rid', 'of', 'not', 'erectile', 'dysfunction']

In [17]:
# Prepare embedding
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [18]:
##Example
#word2vec.vocab['egypt']
#'egypt' not in word2vec.vocab
#word2vec.vocab['of']
'of' not in word2vec.vocab

True

In [19]:
#Example
len(word2vec['egypt'])

300

In [20]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding

In [21]:
dataset = pd.DataFrame()
# Iterate through the text of both questions of the row
#for question in questions_cols:
vocabulary = dict()
inverse_vocabulary = ['<unk>']
q2n = []  # q2n -> question numbers representation
for word in text_to_word_list("How zeinab shouldn do I out get rid of n't zeinab Erectile Dysfunction?"):
  # Check for unwanted words
  if word in stops and word not in word2vec.vocab:
      print('Stop word : ',word)
      continue

  if word not in vocabulary:
      print('Other word : ',word)
      vocabulary[word] = len(inverse_vocabulary)
      q2n.append(len(inverse_vocabulary))
      inverse_vocabulary.append(word)
  else:
      print('else word : ',word)
      q2n.append(vocabulary[word])

    # Replace questions as word to question as number representation
    # print(0)
    # print(question)
    # print(q2n)
print(vocabulary)
print(q2n)

Other word :  how
Other word :  zeinab
Other word :  shouldn
Other word :  do
Other word :  i
Other word :  out
Other word :  get
Other word :  rid
Stop word :  of
Other word :  not
else word :  zeinab
Other word :  erectile
Other word :  dysfunction
{'how': 1, 'zeinab': 2, 'shouldn': 3, 'do': 4, 'i': 5, 'out': 6, 'get': 7, 'rid': 8, 'not': 9, 'erectile': 10, 'dysfunction': 11}
[1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 10, 11]


In [22]:
# Example
word2vec['egypt']
similarity = word2vec.similarity('egypt', 'car')
similarity

0.071321554

In [23]:
from tqdm import tqdm

vocabulary = dict()
inverse_vocabulary = ['<unk>']  # '<unk>' will never be used, it is only a placeholder for the [0, 0, ....0] embedding
questions_cols = ['question1', 'question2']
# Iterate over the questions only of both training and test datasets
for dataset in tqdm([train_df, test_df]):
    for index, row in dataset.iterrows():

        # Iterate through the text of both questions of the row
        for question in questions_cols:

            q2n = []  # q2n -> question numbers representation
            for word in text_to_word_list(row[question]):

                # Check for unwanted words
                if word in stops and word not in word2vec.vocab:
                    continue

                if word not in vocabulary:
                    vocabulary[word] = len(inverse_vocabulary)
                    q2n.append(len(inverse_vocabulary))
                    inverse_vocabulary.append(word)
                else:
                    q2n.append(vocabulary[word])
            # Replace questions as word to question as number representation
            dataset.at[index, question]=q2n            

100%|██████████| 2/2 [14:06<00:00, 423.21s/it]


In [24]:
dataset

,test_id,question1,question2
0,0,"[26, 76, 3, 1663, 1237, 5472, 745, 149, 175, 4...","[41, 330, 3863, 331, 1580, 12991, 212, 1580, 6..."
1,1,"[84, 16, 401, 942, 11093, 225, 833, 54, 26, 21...","[26, 214, 534, 76, 942, 11093, 1864]"
2,2,"[1, 1220, 2, 3, 195, 250, 1096, 251, 91, 817, ...","[1, 99, 1096, 251, 817]"
3,3,"[56, 818, 212, 31956]","[1, 388, 11099]"
4,4,"[26, 86002, 1221, 1204]","[26, 290, 27, 16, 1221, 1204]"
...,...,...,...
3563470,2345791,"[26, 97, 24312, 1245, 1309, 70, 41, 330, 11044...","[1, 2, 3, 278, 169, 6064, 8, 12485, 24312]"
3563471,2345792,"[1, 76, 36, 8, 6214, 155, 161, 10633, 5823]","[26, 213, 10633, 7137, 4126]"
3563472,2345793,"[1, 115, 116, 3484, 13760, 5876, 9728, 1014, 1...","[27, 1014, 15, 9728, 5786, 36, 2793, 5783]"
3563473,2345794,"[1, 653, 3, 195, 4057, 117, 79, 1203, 6214, 8,...","[1, 115, 3, 195, 4057, 117, 9785, 1203, 6214, ..."


In [25]:
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

# Build the embedding matrix
for word, index in vocabulary.items():
    if word in word2vec.vocab:
        embeddings[index] = word2vec.word_vec(word)

del word2vec

In [26]:
embeddings.shape

(121320, 300)

In [27]:
max_seq_length = max(train_df.question1.map(lambda x: len(x)).max(),
                     train_df.question2.map(lambda x: len(x)).max(),
                     test_df.question1.map(lambda x: len(x)).max(),
                     test_df.question2.map(lambda x: len(x)).max())

max_seq_length

213

In [29]:
embedding_layer = Embedding(len(embeddings), embedding_dim, weights=[embeddings], input_length=max_seq_length, trainable=False)